<a href="https://colab.research.google.com/github/mdfahad999/Experiments/blob/main/LLama_index_qna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Get the HTML content of the website
url = "https://science.nasa.gov/astrophysics"
response = requests.get(url)
html_content = response.text

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all the articles on the website
articles = soup.find_all("div", class_="feature-items")
page_text = soup.get_text(separator=" ", strip=True)
# Create a DataFrame to store the data
df = pd.DataFrame(columns=["Title", "Summary", "Link"])

# Iterate over the articles and extract the title, summary, and link
for article in articles:
    title = article.find("h2").text
    summary = article.find("p").text
    link = article.find("a")["href"]

    # Add the data to the DataFrame
    df = df.append({"Title": title, "Summary": summary, "Link": link}, ignore_index=True)

# Print the DataFrame
print(df)

Empty DataFrame
Columns: [Title, Summary, Link]
Index: []


In [ ]:
!pip freeze | grep torch

torch @ https://download.pytorch.org/whl/cu121/torch-2.1.0%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=0d4e8c52a1fcf5ed6cfc256d9a370fcf4360958fc79d0b08a51d55e70914df46
torchaudio @ https://download.pytorch.org/whl/cu121/torchaudio-2.1.0%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=676bda4042734eda99bc59b2d7f761f345d3cde0cad492ad34e3aefde688c6d8
torchdata==0.7.0
torchsummary==1.5.1
torchtext==0.16.0
torchvision @ https://download.pytorch.org/whl/cu121/torchvision-0.16.0%2Bcu121-cp310-cp310-linux_x86_64.whl#sha256=e76e78d0ad43636c9884b3084ffaea8a8b61f21129fbfa456a5fe734f0affea9


In [ ]:
page_text

"NASA Astrophysics - NASA Science Explore Search Submit News & Events Multimedia NASA+ Home NASA Astrophysics Overview Big Questions Organization and Staff Focus Areas Dark Energy, Dark Matter Black Holes The Big Bang Galaxies Stars Exoplanets → Programs Programs Overview Physics of the Cosmos Cosmic Origins Exoplanet Exploration Explorers Astrophysics Research Astrophysics Pioneers Astrophysics Division Technology Great Observatory Maturation Program Astrophysics Strategic Missions Program Data Resources Documents 2020 Decadal Survey APD Responses to Decadal Pre 2020 Decadal Reports Fleet Chart The Universe Explore This Section Overview Focus Areas Programs Data Resources The Universe NASA Astrophysics In the Science Mission Directorate (SMD), the Astrophysics Division studies the universe. The universe is everything. It includes all of space, matter, energy, time, and you. NASA’s Astrophysics Division is dedicated to exploring the universe, pushing the boundaries of what is known of 

In [ ]:
# !pip install llama-index langchain openai FlagEmbedding==1.1.5

In [ ]:
!pip install streamlit
!pip install streamlit-pills
!pip install llama-index==0.9.7
# !pip install llama-index==0.9.21
!pip install llama-hub==0.0.44
# NOTE: this is due to a trivial dependency in the web tool, will refactor
!pip install langchain==0.0.305
!pip install pypdf
!pip install FlagEmbedding==1.1.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.3/706.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 900.5/900.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 24.5 MB/s 

In [ ]:


from llama_index.llms import OpenAI
# from llama_index.llms.base import LLM
from llama_index.llms.utils import resolve_llm
from pydantic import BaseModel, Field
import os
from llama_index.agent import OpenAIAgent, ReActAgent
from llama_index.agent.react.prompts import REACT_CHAT_SYSTEM_HEADER
from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    ServiceContext,
    Document,
)
from typing import List, cast, Optional
from llama_index import SimpleDirectoryReader
from llama_index.embeddings.utils import resolve_embed_model
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.types import BaseAgent
from llama_index.chat_engine.types import BaseChatEngine
from llama_index.agent.react.formatter import ReActChatFormatter
from llama_index.llms.openai_utils import is_function_calling_model
from llama_index.chat_engine import CondensePlusContextChatEngine
# from core.builder_config import BUILDER_LLM
from typing import Dict, Tuple, Any
import streamlit as st

from llama_index.callbacks import CallbackManager, trace_method
# from core.callback_manager import StreamlitFunctionsCallbackHandler
from llama_index.schema import ImageNode, NodeWithScore

### BETA: Multi-modal
from llama_index.indices.multi_modal.base import MultiModalVectorStoreIndex
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.indices.multi_modal.retriever import (
    MultiModalVectorIndexRetriever,
)
from llama_index.llms import ChatMessage
from llama_index.query_engine.multi_modal import SimpleMultiModalQueryEngine
from llama_index.chat_engine.types import (
    AGENT_CHAT_RESPONSE_TYPE,
    StreamingAgentChatResponse,
    AgentChatResponse,
)
from llama_index.llms.base import ChatResponse
from typing import Generator

In [ ]:
import llama_index
print(llama_index.__version__)

0.9.7


In [ ]:
from llama_index.llms import OpenAI
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
import openai
import json
from pydantic import BaseModel
from llama_index.llms.openai_utils import to_openai_tool

class Song(BaseModel):
    """A song with name and artist"""

    name: str
    code: str


song_fn = to_openai_tool(Song)
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxx"



llm = OpenAI(temperature=0.1, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)
try:
    response = OpenAI(temperature=0.1, model="gpt-4").complete("Write a code  for longest common substring", tools=[song_fn])
except Exception as e:
        print(e)
print(response.text)
dictionary={'code':response.text}
with open('code.json','w') as f:
    json.dump(dictionary, f)

Sure, here is a Python code for finding the longest common substring between two strings.

```python
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for i in range(1 + len(s1))]
    longest, x_longest = 0, 0
    for x in range(1, 1 + len(s1)):
        for y in range(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
            else:
                m[x][y] = 0
    return s1[x_longest - longest: x_longest]

# Test the function
s1 = "ABCDEF"
s2 = "ZBCDFG"
print(longest_common_substring(s1, s2))  # Outputs: "BCD"
```

This function uses dynamic programming to find the longest common substring. It creates a 2D array `m` where the cell `m[x][y]` contains the length of the longest common substring ending at positions `x` and `y` in `s1` and `s2` respectively. It then iterates over `s1` and `s2`, updating `m` and k

In [ ]:
# llm.complete("how horror is insedious movie").text

In [ ]:

import os
import torch
from FlagEmbedding import FlagModel

# see if GPU is available for model to use
device = "0" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = device
print(device)

embed_model = FlagModel(
    'BAAI/bge-large-en-v1.5',
    query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
    use_fp16=True  # did not note noticable difference between fp16 or fp32
)

def embed(docs: list[str]):
    doc_embeds = embed_model.encode(docs)
    return doc_embeds

0


In [ ]:
embed(['Hi how are you'])

array([[ 0.0682  , -0.006775, -0.03925 , ..., -0.03394 , -0.0341  ,
         0.00914 ]], dtype=float16)

In [ ]:
embed_model.model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-23): 24 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inpl

In [ ]:
# import streamlit as st
# import os

# ### DEFINE BUILDER_LLM #####
# ## Uncomment the LLM you want to use to construct the meta agent

# ## OpenAI
# from llama_index.llms import OpenAI

# # set OpenAI Key - use Streamlit secrets
# os.environ["OPENAI_API_KEY"] = st.secrets.openai_key
# # load LLM

import openai
openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"



llm = OpenAI(temperature=0.1, model="gpt-4")

BUILDER_LLM = OpenAI(model="gpt-4-1106-preview")


class RAGParams(BaseModel):
    """RAG parameters.

    Parameters used to configure a RAG pipeline.

    """

    include_summarization: bool = Field(
        default=False,
        description=(
            "Whether to include summarization in the RAG pipeline. (only for GPT-4)"
        ),
    )
    top_k: int = Field(
        default=2, description="Number of documents to retrieve from vector store."
    )
    chunk_size: int = Field(default=1024, description="Chunk size for vector store.")
    embed_model: str = Field(
        default="default", description="Embedding model to use (default is OpenAI)"
    )
    llm: str = Field(
        default="gpt-4-1106-preview", description="LLM to use for summarization."
    )


def _resolve_llm(llm_str: str) :
    """Resolve LLM."""
    # TODO: make this less hardcoded with if-else statements
    # see if there's a prefix
    # - if there isn't, assume it's an OpenAI model
    # - if there is, resolve it
    tokens = llm_str.split(":")
    if len(tokens) == 1:
        os.environ["OPENAI_API_KEY"] = st.secrets.openai_key
        llm: LLM = OpenAI(model=llm_str)
    elif tokens[0] == "local":
        llm = resolve_llm(llm_str)
    elif tokens[0] == "openai":
        os.environ["OPENAI_API_KEY"] = st.secrets.openai_key
        llm = OpenAI(model=tokens[1])
    elif tokens[0] == "anthropic":
        os.environ["ANTHROPIC_API_KEY"] = st.secrets.anthropic_key
        llm = Anthropic(model=tokens[1])
    elif tokens[0] == "replicate":
        os.environ["REPLICATE_API_KEY"] = st.secrets.replicate_key
        llm = Replicate(model=tokens[1])
    else:
        raise ValueError(f"LLM {llm_str} not recognized.")
    return llm


def load_data(
    file_names: Optional[List[str]] = None,
    directory: Optional[str] = None,
    urls: Optional[List[str]] = None,
) :
    """Load data."""
    file_names = file_names or []
    directory = directory or ""
    urls = urls or []

    # get number depending on whether specified
    num_specified = sum(1 for v in [file_names, urls, directory] if v)

    if num_specified == 0:
        raise ValueError("Must specify either file_names or urls or directory.")
    elif num_specified > 1:
        raise ValueError("Must specify only one of file_names or urls or directory.")
    elif file_names:
        reader = SimpleDirectoryReader(input_files=file_names)
        docs = reader.load_data()
    elif directory:
        reader = SimpleDirectoryReader(input_dir=directory)
        docs = reader.load_data()
    elif urls:
        from llama_hub.web.simple_web.base import SimpleWebPageReader

        # use simple web page reader from llamahub
        loader = SimpleWebPageReader()
        docs = loader.load_data(urls=urls)
    else:
        raise ValueError("Must specify either file_names or urls or directory.")

    return docs


def load_agent(
    tools: List,
    llm,
    system_prompt: str,
    extra_kwargs: Optional[Dict] = None,
    **kwargs: Any,
) :
    """Load agent."""
    extra_kwargs = extra_kwargs or {}
    if isinstance(llm, OpenAI) and is_function_calling_model(llm.model):
        # TODO: use default msg handler
        # TODO: separate this from agent_utils.py...
        def _msg_handler(msg: str) -> None:
            """Message handler."""
            st.info(msg)
            st.session_state.agent_messages.append(
                {"role": "assistant", "content": msg, "msg_type": "info"}
            )

        # add streamlit callbacks (to inject events)
        # handler = StreamlitFunctionsCallbackHandler(_msg_handler)
        # callback_manager = CallbackManager([handler])
        # get OpenAI Agent
        agent: BaseChatEngine = OpenAIAgent.from_tools(
            tools=tools,
            llm=llm,
            system_prompt=system_prompt,
            **kwargs,
            callback_manager=callback_manager,
        )
    else:
        if "vector_index" not in extra_kwargs:
            raise ValueError(
                "Must pass in vector index for CondensePlusContextChatEngine."
            )
        vector_index = cast(VectorStoreIndex, extra_kwargs["vector_index"])
        rag_params = cast(RAGParams, extra_kwargs["rag_params"])
        # use condense + context chat engine
        agent = CondensePlusContextChatEngine.from_defaults(
            vector_index.as_retriever(similarity_top_k=rag_params.top_k),
        )

    return agent





def load_meta_agent(
    tools: List,
    llm,
    system_prompt: str,
    extra_kwargs: Optional[Dict] = None,
    **kwargs: Any,
) :
    """Load meta agent.

    TODO: consolidate with load_agent.

    The meta-agent *has* to perform tool-use.

    """
    extra_kwargs = extra_kwargs or {}
    if isinstance(llm, OpenAI) and is_function_calling_model(llm.model):
        # get OpenAI Agent

        agent: BaseAgent = OpenAIAgent.from_tools(
            tools=tools,
            llm=llm,
            system_prompt=system_prompt,
            **kwargs,
        )
    else:
        agent = ReActAgent.from_tools(
            tools=tools,
            llm=llm,
            react_chat_formatter=ReActChatFormatter(
                system_header=system_prompt + "\n" + REACT_CHAT_SYSTEM_HEADER,
            ),
            **kwargs,
        )

    return agent




def construct_agent(
    system_prompt: str,
    rag_params: RAGParams,
    docs: List[Document],
    vector_index: Optional[VectorStoreIndex] = None,
    additional_tools: Optional[List] = None,
) -> Tuple[BaseChatEngine, Dict]:
    """Construct agent from docs / parameters / indices."""
    extra_info = {}
    additional_tools = additional_tools or []

    # first resolve llm and embedding model
    embed_model = resolve_embed_model(rag_params.embed_model)
    # llm = resolve_llm(rag_params.llm)
    # TODO: use OpenAI for now
    # llm = OpenAI(model=rag_params.llm)
    llm = _resolve_llm(rag_params.llm)

    # first let's index the data with the right parameters
    service_context = ServiceContext.from_defaults(
        chunk_size=rag_params.chunk_size,
        llm=llm,
        embed_model=embed_model,
    )

    if vector_index is None:
        vector_index = VectorStoreIndex.from_documents(
            docs, service_context=service_context
        )
    else:
        pass

    extra_info["vector_index"] = vector_index

    vector_query_engine = vector_index.as_query_engine(
        similarity_top_k=rag_params.top_k
    )
    all_tools = []
    vector_tool = QueryEngineTool(
        query_engine=vector_query_engine,
        metadata=ToolMetadata(
            name="vector_tool",
            description=("Use this tool to answer any user question over any data."),
        ),
    )
    all_tools.append(vector_tool)
    if rag_params.include_summarization:
        summary_index = SummaryIndex.from_documents(
            docs, service_context=service_context
        )
        summary_query_engine = summary_index.as_query_engine()
        summary_tool = QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    "Use this tool for any user questions that ask "
                    "for a summarization of content"
                ),
            ),
        )
        all_tools.append(summary_tool)

    # then we add tools
    all_tools.extend(additional_tools)

    # build agent
    if system_prompt is None:
        return "System prompt not set yet. Please set system prompt first."

    agent = load_agent(
        all_tools,
        llm=llm,
        system_prompt=system_prompt,
        verbose=True,
        extra_kwargs={"vector_index": vector_index, "rag_params": rag_params},
    )
    return agent, extra_info


def get_web_agent_tool() :
    """Get web agent tool.

    Wrap with our load and search tool spec.

    """
    from llama_hub.tools.metaphor.base import MetaphorToolSpec

    # TODO: set metaphor API key
    metaphor_tool = MetaphorToolSpec(
        api_key=st.secrets.metaphor_key,
    )
    metaphor_tool_list = metaphor_tool.to_tool_list()

    # TODO: LoadAndSearch doesn't work yet
    # The search_and_retrieve_documents tool is the third in the tool list,
    # as seen above
    # wrapped_retrieve = LoadAndSearchToolSpec.from_defaults(
    #     metaphor_tool_list[2],
    # )

    # NOTE: requires openai right now
    # We don't give the Agent our unwrapped retrieve document tools
    # instead passing the wrapped tools
    web_agent = OpenAIAgent.from_tools(
        # [*wrapped_retrieve.to_tool_list(), metaphor_tool_list[4]],
        metaphor_tool_list,
        llm=BUILDER_LLM,
        verbose=True,
    )

    # return agent as a tool
    # TODO: tune description
    web_agent_tool = QueryEngineTool.from_defaults(
        web_agent,
        name="web_agent",
        description="""
            This agent can answer questions by searching the web. \
Use this tool if the answer is ONLY likely to be found by searching \
the internet, especially for queries about recent events.
        """,
    )

    return web_agent_tool


def get_tool_objects(tool_names: List[str]) -> List:
    """Get tool objects from tool names."""
    # construct additional tools
    tool_objs = []
    for tool_name in tool_names:
        if tool_name == "web_search":
            # build web agent
            tool_objs.append(get_web_agent_tool())
        else:
            raise ValueError(f"Tool {tool_name} not recognized.")

    return tool_objs


In [ ]:
[os.path.join(os.getcwd(),x)for x in os.listdir('.')  if x.endswith('.pdf')]

['/content/a_history_of_near-earth_object_research_tagged.pdf']

In [ ]:
data=load_data(file_names=[os.path.join(os.getcwd(),x)for x in os.listdir('.')  if x.endswith('.pdf')])


In [ ]:
len(data)

407

In [ ]:

for dt in data:
    print(dt.text)
    print('\n\n\n')
    print(dt.metadata)
    print("====")

Erik M. Conway  • Donald K. Yeomans
Meg Rosenburg
A HISTORY OF
NEAR-EARTH 
OBJECTS  
RESEARCH





{'page_label': 'a', 'file_name': 'a_history_of_near-earth_object_research_tagged.pdf', 'file_path': '/content/a_history_of_near-earth_object_research_tagged.pdf', 'file_type': 'application/pdf', 'file_size': 4556385, 'creation_date': '2023-12-25', 'last_modified_date': '2023-12-25', 'last_accessed_date': '2023-12-25'}
====





{'page_label': 'b', 'file_name': 'a_history_of_near-earth_object_research_tagged.pdf', 'file_path': '/content/a_history_of_near-earth_object_research_tagged.pdf', 'file_type': 'application/pdf', 'file_size': 4556385, 'creation_date': '2023-12-25', 'last_modified_date': '2023-12-25', 'last_accessed_date': '2023-12-25'}
====
A HISTORY OF
NEAR-EARTH  
OBJECTS 
RESEARCH




{'page_label': 'i', 'file_name': 'a_history_of_near-earth_object_research_tagged.pdf', 'file_path': '/content/a_history_of_near-earth_object_research_tagged.pdf', 'file_type': 'application/pdf', 'fi

In [ ]:


from llama_index import ServiceContext, LLMPredictor,  PromptHelper,VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter,SentenceSplitter
from llama_index import StorageContext, load_index_from_storage
from llama_index.node_parser import SimpleNodeParser
import tiktoken
import os
import openai
import logging
import sys
import nltk
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.node_parser import SentenceWindowNodeParser
import shutil
import pandas as pd
path='./storage'
try:

    shutil.rmtree(path)
except:

    pass

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
class Text:
    BOLD_START = '\033[1m'
    END = '\033[0m'
    UNDERLINE = '\033[4m'
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'


openai.api_key = "sxxxxxxxxxxxxxxxxxxxxx"

synthesis_model="gpt-4"
llm = OpenAI(temperature=0.1, model=synthesis_model)

from llama_index.embeddings import HuggingFaceEmbedding
embmodel_id='BAAI/bge-small-en-v1.5'

embed_model = HuggingFaceEmbedding(model_name=embmodel_id)

# text_splitter = TokenTextSplitter(
#   separator=" ",
#   chunk_size=1024,
#   chunk_overlap=20,
#   backup_separators=["\n"],
# )
text_splitter = SentenceSplitter(chunk_size=512, chunk_overlap=10)


node_parser = SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)
# node_parser = SimpleNodeParser.from_defaults(
#   text_splitter=text_splitter
# )
prompt_helper = PromptHelper(
  context_window=4096,
  num_output=256,
  chunk_overlap_ratio=0.1,
  chunk_size_limit=None
)

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model=embed_model,
  node_parser=node_parser,
  prompt_helper=prompt_helper
)
# print(service_context)
# documents = data
# index = VectorStoreIndex.from_documents(
#     documents,
#     service_context = service_context
#     )

if not os.path.exists("./storage"):
    print(Text.BOLD_START+Text.PURPLE +'Index is not Present .Creating it!!!!!!'+Text.END )
    # load the documents and create the index
    documents = data
    index = VectorStoreIndex.from_documents(
    documents,
    service_context = service_context,show_progress=True
    )
    # store it for later
    index.storage_context.persist()
else:
    print(Text.BOLD_START+Text.GREEN +'Index already exist' +Text.END )
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir="./storage")
    index = load_index_from_storage(storage_context)

# index.storage_context.persist()

query_engine = index.as_query_engine()

Questions ="When did NASA  established the Planetary Defense Coordination Office?"

if not isinstance(Questions,list):
    Questions =[Questions]

data=[]
for Question in Questions:
    response = query_engine.query(Question)
    final_response = {}
    final_response['Query'] =Question
    final_response['Result'] = response.response
    final_response['Filename'] = list(set([ v['file_name'] for x ,v in response.metadata.items()]))
    final_response['page_no'] = [ v['page_label'] for x ,v in response.metadata.items()]
    data.append(final_response)
print(final_response)

df =pd.DataFrame(data)
embmodel_id =embmodel_id.replace('/','_')
df.to_csv(f'LLama_index_{embmodel_id}_{synthesis_model}.csv', index=False)

Streaming output truncated to the last 5000 lines.



{'page_label': '312', 'file_name': 'a_history_of_near-earth_object_research_tagged.pdf', 'file_path': '/content/a_history_of_near-earth_object_research_tagged.pdf', 'file_type': 'application/pdf', 'file_size': 4556385, 'creation_date': '2023-12-25', 'last_modified_date': '2023-12-25', 'last_accessed_date': '2023-12-25'}
====
Chapter 11: Conclusion 313
been extended from the visible into the near-infrared region of the spectrum, 
and thanks largely to the space-based NEOWISE satellite, there are now over 
1300 NEOs and 39,475 solar system objects for which infrared data (and 
hence accurate diameters and albedos) are available.11
Radar observations of NEOs are now often capable of “imaging” NEOs to 
better than 4-meter resolution, thus achieving a level of spatial characteriza -
tion that is only exceeded by some spacecraft observations. There have been 
nine comets and over a dozen asteroids visited by spacecraft and dust samples 
r

Parsing nodes:   0%|          | 0/407 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/6514 [00:00<?, ?it/s]

{'Query': 'When did NASA  established the Planetary Defense Coordination Office?', 'Result': 'NASA established the Planetary Defense Coordination Office on 7 January 2016.', 'Filename': ['a_history_of_near-earth_object_research_tagged.pdf'], 'page_no': ['1', '286']}


In [ ]:
final_response

{'Query': 'When did NASA  established the Planetary Defense Coordination Office?',
 'Result': 'NASA established the Planetary Defense Coordination Office on 7 January 2016.',
 'Filename': ['a_history_of_near-earth_object_research_tagged.pdf'],
 'page_no': ['1', '286']}

In [ ]:
pwd

'/content'

In [ ]:
import os

os.listdir('/content')

['.config', 'sample_data']

In [ ]:
## LLama_index class  for Question answering

from llama_index import ServiceContext, LLMPredictor,  PromptHelper,VectorStoreIndex
from llama_index.llms import OpenAI
from llama_index.text_splitter import TokenTextSplitter,SentenceSplitter
from llama_index import StorageContext, load_index_from_storage
from llama_index.node_parser import SimpleNodeParser
import tiktoken
import os
import openai
import logging
import sys
import nltk
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.embeddings import HuggingFaceEmbedding
import shutil
from llama_index.llms import OpenAI
# from llama_index.llms.base import LLM
from llama_index.llms.utils import resolve_llm
from pydantic import BaseModel, Field
import os
from llama_index.agent import OpenAIAgent, ReActAgent
from llama_index.agent.react.prompts import REACT_CHAT_SYSTEM_HEADER
from llama_index import (
    VectorStoreIndex,
    SummaryIndex,
    ServiceContext,
    Document,
)

from typing import List, cast, Optional
from llama_index import SimpleDirectoryReader
from llama_index.embeddings.utils import resolve_embed_model
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.agent.types import BaseAgent
from llama_index.chat_engine.types import BaseChatEngine
from llama_index.agent.react.formatter import ReActChatFormatter
from llama_index.llms.openai_utils import is_function_calling_model
from llama_index.chat_engine import CondensePlusContextChatEngine
# from core.builder_config import BUILDER_LLM
from typing import Dict, Tuple, Any
import pandas as pd
path='./storage'
try:

    shutil.rmtree(path)
except:
    pass

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))
class Text:
    BOLD_START = '\033[1m'
    END = '\033[0m'
    UNDERLINE = '\033[4m'
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'


openai.api_key = "sk-Occccccccccccccccccccccccccccccc"


synthesis_model_id ='gpt4'


embmodel_id='BAAI/bge-small-en-v1.5'

def synthesis_model_fn(synthesis_model_id:str=None):
    #synthesis_model="gpt-4"
    llm = OpenAI(temperature=0.1, model=synthesis_model_id)
    return llm

def embedding_model_fn(embmodel_id:str=None):
    #embmodel_id='BAAI/bge-small-en-v1.5'
    embed_model = HuggingFaceEmbedding(model_name=embmodel_id)
    return embed_model

def node_parser_func():
    return SentenceWindowNodeParser.from_defaults(
    # how many sentences on either side to capture
    window_size=3,
    # the metadata key that holds the window of surrounding sentences
    window_metadata_key="window",
    # the metadata key that holds the original sentence
    original_text_metadata_key="original_sentence",
)
def prompt_helper_fnc():
    print(type(PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None)))
    return PromptHelper(context_window=4096, num_output=256, chunk_overlap_ratio=0.1, chunk_size_limit=None)


class LLama_Index_qna:
    def __init__(self,synthesis_model=None,embedding_model=None, text_splitter=None,node_parser=None,PromptHelper=None):
        self.synthesis_model =synthesis_model_fn(synthesis_model_id)
        self.embedding_model =embedding_model_fn(embmodel_id)
        print(self.embedding_model)
        self.text_splitter =SentenceSplitter(chunk_size=512, chunk_overlap=10)
        self.node_parser=node_parser_func()
        print(self.node_parser)
        self.prompt_helper =prompt_helper_fnc
        self.service_context = ServiceContext.from_defaults(llm=self.synthesis_model,embed_model=self.embedding_model,node_parser=self.node_parser,prompt_helper=self.prompt_helper)
        self.persist_directory ='./storage'
        print(self.service_context)

    @staticmethod
    def load_data(
        file_names: Optional[List[str]] = None,
        directory: Optional[str] = None,
        urls: Optional[List[str]] = None,
    ) :
        """Load data."""
        file_names = file_names or []
        directory = directory or ""
        urls = urls or []

        # get number depending on whether specified
        num_specified = sum(1 for v in [file_names, urls, directory] if v)

        if num_specified == 0:
            raise ValueError("Must specify either file_names or urls or directory.")
        elif num_specified > 1:
            raise ValueError("Must specify only one of file_names or urls or directory.")
        elif file_names:
            reader = SimpleDirectoryReader(input_files=file_names)
            docs = reader.load_data()
        elif directory:
            reader = SimpleDirectoryReader(input_dir=directory)
            docs = reader.load_data()
        elif urls:
            from llama_hub.web.simple_web.base import SimpleWebPageReader

            # use simple web page reader from llamahub
            loader = SimpleWebPageReader()
            docs = loader.load_data(urls=urls)
        else:
            raise ValueError("Must specify either file_names or urls or directory.")

        return docs

    def creating_index(self,file_names=None):
        if file_names is None:
            try:
                file_names= [os.path.join(os.getcwd(),x)for x in os.listdir('.')  if x.endswith('.pdf')]
            except:
                raise ValueError(Text.BOLD_START+Text.RED +"May be file is not uplloaded!!!!"+Text.END )

        if not os.path.exists(self.persist_directory):

            print(Text.BOLD_START+Text.PURPLE +'Index is not Present .Creating it!!!!!!'+Text.END )
            # load the documents and create the index
            print(file_names)
            documents =self.load_data(file_names=file_names)
            print('Doc  :',documents)
            index = VectorStoreIndex.from_documents(
            documents,
            service_context = self.service_context,
            # show_progress=True
            )
            # store it for later
            return index.storage_context.persist(persist_dir=self.persist_directory)
        else:
            print(Text.BOLD_START+Text.GREEN +'Index already exist' +Text.END )
            # load the existing index
            try:
                print()
                storage_context = StorageContext.from_defaults(persist_dir=self.persist_directory)
                index = load_index_from_storage(storage_context)
                return index
            except:
                print('Getting Error in loading from persist dir, with dimension mismatch')
                documents =self.load_data(file_names=file_names)
                index = VectorStoreIndex.from_documents(
                documents,
                service_context = self.service_context
                )
                # store it for later
                return index.storage_context.persist(persist_dir=self.persist_directory)

        def qna(self,query:str=None):
            print('NN')
            index =self.creating_index()
            query_engine = index.as_query_engine()
            if not isinstance(query,list):
                Questions =[query]
            data=[]
            for Question in Questions:
                response = query_engine.query(Question)
                final_response = {}
                final_response['Query'] =Question
                final_response['Result'] = response.response
                final_response['Filename'] = list(set([ v['file_name'] for x ,v in response.metadata.items()]))
                final_response['page_no'] = [ v['page_label'] for x ,v in response.metadata.items()]
                data.append(final_response)
            df =pd.DataFrame(data)
            embmodel_id =embmodel_id.replace('/','_')
            df.to_csv(f'Results/LLama_index_{embmodel_id}_{synthesis_model_id}.csv', index=False)
            #print(final_response)
            return final_reponse
# text_splitter = TokenTextSplitter(
#   separator=" ",
#   chunk_size=1024,
#   chunk_overlap=20,
#   backup_separators=["\n"],
# )



# node_parser = SimpleNodeParser.from_defaults(
#   text_splitter=text_splitter
# )


# documents = data
# index = VectorStoreIndex.from_documents(
#     documents,
#     service_context = service_context
#     )


# index.storage_context.persist()


# from llama_index import Prompt

# Define a custom prompt
# template = (
#     "We have provided context information below. \n"
#     "---------------------\n"
#     "{context_str}"
#     "\n---------------------\n"
#     "Given this information, please answer the question and each answer should start with code word AI Demos: {query_str}\n"
# )
# qa_template = Prompt(template)

# # Use the custom prompt when querying
# query_engine = custom_llm_index.as_query_engine(text_qa_template=qa_template)

In [ ]:
llama_obj=LLama_Index_qna()
llama_obj.creating_index()
llama_obj.qna('when did NASA  established the Planetary Defense Coordination Office?')

model_name='BAAI/bge-small-en-v1.5' embed_batch_size=10 callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x78e2024f4f70> tokenizer_name='BAAI/bge-small-en-v1.5' max_length=512 pooling=<Pooling.CLS: 'cls'> normalize=True query_instruction=None text_instruction=None cache_folder=None
include_metadata=True include_prev_next_rel=True callback_manager=<llama_index.callbacks.base.CallbackManager object at 0x78e2024f4d30> sentence_splitter=<function split_by_sentence_tokenizer.<locals>.split at 0x78e2024f85e0> window_size=3 window_metadata_key='window' original_text_metadata_key='original_sentence'
ServiceContext(llm_predictor=LLMPredictor(system_prompt=None, query_wrapper_prompt=None, pydantic_program_mode=<PydanticProgramMode.DEFAULT: 'default'>), prompt_helper=<function prompt_helper_fnc at 0x78e217539fc0>, embed_model=HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5', embed_batch_size=10, callback_manager=<llama_index.callbacks.base.CallbackManager object at 0